In [20]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# MySQL 연동
from sqlalchemy import create_engine
import pymysql

import warnings
warnings.filterwarnings("ignore")

In [22]:
from sqlalchemy import create_engine

DB_USER = "root"
DB_PASSWORD = "fls0413!"
DB_HOST = "localhost"
DB_PORT = 3306
DB_NAME = "core5_ems"

engine = create_engine(
    f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}",
    echo=False
)

DB 생성 (최초 1회)

In [7]:
from sqlalchemy import create_engine

engine_tmp = create_engine(
    f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}",
    echo=False
)

with engine_tmp.connect() as conn:
    conn.exec_driver_sql("CREATE DATABASE IF NOT EXISTS core5_ems")

Core 4 결과 CSV 로드

In [11]:
PATH_AMI = "../data_csv/amiwea.csv"
PATH_REACTIVE = "../data_csv/core4_ipynb2_reactive_ts.csv"
PATH_PROACTIVE = "../data_csv/core4_ipynb2_proactive_ts.csv"

# AMI: timestamp 존재
df_ami = pd.read_csv(PATH_AMI, parse_dates=["timestamp"]).sort_values("timestamp").set_index("timestamp")

# ipynb2 reactive/proactive: DATE_TIME 존재 (timestamp 아님)
df_re = pd.read_csv(PATH_REACTIVE)
df_re["DATE_TIME"] = pd.to_datetime(df_re["DATE_TIME"], errors="coerce")
df_re = df_re.dropna(subset=["DATE_TIME"]).sort_values("DATE_TIME").set_index("DATE_TIME")

df_pr = pd.read_csv(PATH_PROACTIVE)
df_pr["DATE_TIME"] = pd.to_datetime(df_pr["DATE_TIME"], errors="coerce")
df_pr = df_pr.dropna(subset=["DATE_TIME"]).sort_values("DATE_TIME").set_index("DATE_TIME")

시간축 정렬

In [13]:
common_idx = df_ami.index.intersection(df_re.index).intersection(df_pr.index)

df_ami = df_ami.loc[common_idx]
df_re = df_re.loc[common_idx]
df_pr = df_pr.loc[common_idx]

EMS–ESS 통합 테이블 구성

In [14]:
df_core5 = pd.DataFrame(index=common_idx)

df_core5["consumption_kw"] = df_ami["consumption"]

df_core5["ess_load_reactive"] = df_re["load_adj_kw"]
df_core5["ess_load_proactive"] = df_pr["load_adj_kw"]

df_core5["ess_soc_reactive"] = df_re["soc_kwh"]
df_core5["ess_soc_proactive"] = df_pr["soc_kwh"]

# IoT placeholder
df_core5["iot_info_density"] = np.nan
df_core5["iot_prediction_confidence"] = np.nan

IoT Feature Placeholder (의료 IoT 매핑용)

In [15]:
df_core5["iot_info_density"] = np.nan
df_core5["iot_prediction_confidence"] = np.nan

MySQL 테이블 저장

In [16]:
df_core5.reset_index().rename(columns={"index": "timestamp"}).to_sql(
    name="ems_ess_core4",
    con=engine,
    if_exists="replace",
    index=False
)

0

페이스메이커.csv 로드 + 기본 확인

In [24]:
PATH_PACEMAKER = "../data_csv/페이스메이커.csv" 

pm = pd.read_csv(PATH_PACEMAKER)
pm.head(), pm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   episode_id            15000 non-null  int64  
 1   step                  15000 non-null  int64  
 2   patient_id            15000 non-null  int64  
 3   age                   15000 non-null  int64  
 4   sex                   15000 non-null  int64  
 5   weight_kg             15000 non-null  float64
 6   diagnosis             15000 non-null  object 
 7   baseline_hr           15000 non-null  float64
 8   egfr                  15000 non-null  float64
 9   beta_blocker_dose_mg  15000 non-null  float64
 10  current_hr            15000 non-null  float64
 11  spo2                  15000 non-null  float64
 12  sbp                   15000 non-null  float64
 13  dbp                   15000 non-null  float64
 14  activity_level        15000 non-null  int64  
 15  arrhythmia_risk    

(   episode_id  step  patient_id  age  sex  weight_kg diagnosis  baseline_hr  \
 0           0     0           0   88    1  61.657439        AF    93.189022   
 1           0     1           0   88    1  61.657439        AF    93.189022   
 2           0     2           0   88    1  61.657439        AF    93.189022   
 3           0     3           0   88    1  61.657439        AF    93.189022   
 4           0     4           0   88    1  61.657439        AF    93.189022   
 
         egfr  beta_blocker_dose_mg  ...  prev_pacing_rate  action  \
 0  79.192128             65.348695  ...                76       0   
 1  79.192128             65.348695  ...                76       0   
 2  79.192128             65.348695  ...                76       1   
 3  79.192128             65.348695  ...                81       0   
 4  79.192128             65.348695  ...                81       0   
 
    new_pacing_rate    next_hr    next_sbp   next_dbp  next_spo2  \
 0               76  87.7560

시간 컬럼 자동 탐지 + timestamp 생성

In [26]:
pm = pm.copy()

# ✅ 이 데이터는 timestamp가 아니라 (episode_id, step)가 시간축
need_cols = ["episode_id", "step"]
for c in need_cols:
    if c not in pm.columns:
        raise ValueError(f"{c} 컬럼이 없음. 현재 컬럼: {list(pm.columns)}")

pm["timestamp"] = (
    pm["episode_id"].astype(str).str.zfill(6)
    + "_"
    + pm["step"].astype(int).astype(str).str.zfill(4)
)

# 정렬 키: (episode_id, step)
pm = pm.sort_values(["episode_id", "step"]).set_index("timestamp")

pm.index.min(), pm.index.max(), pm.shape

('000000_0000', '000499_0029', (15000, 26))

시간 단위 정렬/리샘플
페이스메이커 데이터가 초/분 단위로 촘촘하면, AMI와 맞추려고 시간 단위로 내려서 씀.

MySQL 적재 (페이스메이커 테이블 별도 저장)

In [33]:
pm.reset_index().to_sql(
    name="pacemaker_iot",
    con=engine,
    if_exists="replace",
    index=False
)

15000

적재 확인

In [34]:
pd.read_sql("SELECT COUNT(*) AS row_count FROM pacemaker_iot", engine)

,row_count
0,15000


Core4 테이블과 “시간 교집합” 조인해서 Core5 통합 테이블 생성

In [35]:
# DB에서 다시 읽어와 조인(가장 안전)
core4 = pd.read_sql("SELECT * FROM ems_ess_core4", engine, parse_dates=["timestamp"]).set_index("timestamp")
pm_db = pd.read_sql("SELECT * FROM pacemaker_iot", engine, parse_dates=["timestamp"]).set_index("timestamp")

common = core4.index.intersection(pm_db.index)

core5 = core4.loc[common].join(pm_db.loc[common], how="inner")

core5.reset_index().to_sql(
    name="core5_ems_ess_iot",
    con=engine,
    if_exists="replace",
    index=False
)

core5.shape

(0, 33)

In [36]:
import pandas as pd

pd.read_sql("SHOW TABLES;", engine)

,Tables_in_core5_ems
0,core5_ems_ess_iot
1,ems_ess_core4
2,pacemaker_iot


In [37]:
pd.read_sql("""
SELECT 
  (SELECT COUNT(*) FROM ems_ess_core4) AS ems_rows,
  (SELECT COUNT(*) FROM pacemaker_iot) AS pacemaker_rows;
""", engine)

,ems_rows,pacemaker_rows
0,0,15000


In [38]:
pd.read_sql("DESCRIBE ems_ess_core4;", engine)

,Field,Type,Null,Key,Default,Extra
0,timestamp,datetime,YES,,None,
1,consumption_kw,double,YES,,None,
2,ess_load_reactive,double,YES,,None,
3,ess_load_proactive,double,YES,,None,
4,ess_soc_reactive,double,YES,,None,
5,ess_soc_proactive,double,YES,,None,
6,iot_info_density,double,YES,,None,
7,iot_prediction_confidence,double,YES,,None,


In [39]:
pd.read_sql("DESCRIBE pacemaker_iot;", engine)

,Field,Type,Null,Key,Default,Extra
0,timestamp,text,YES,,None,
1,episode_id,bigint,YES,,None,
2,step,bigint,YES,,None,
3,patient_id,bigint,YES,,None,
4,age,bigint,YES,,None,
5,sex,bigint,YES,,None,
6,weight_kg,double,YES,,None,
7,diagnosis,text,YES,,None,
8,baseline_hr,double,YES,,None,
9,egfr,double,YES,,None,


In [42]:
# df_ami가 비어있는지 먼저 확인
print("df_ami shape:", df_ami.shape)

# timestamp가 index인지 확인
print("index name:", df_ami.index.name)

# index → 컬럼으로 명확히 변환
df_ami_out = df_ami.reset_index()

# 컬럼 이름 확인
print(df_ami_out.columns)

# DB 적재
df_ami_out.to_sql(
    name="ems_ess_core4",
    con=engine,
    if_exists="replace",
    index=False
)

df_ami shape: (0, 6)
index name: None
Index(['index', 'household_id', 'consumption', 'Season', 'temperature',
       'humidity', 'windSpeed'],
      dtype='object')


0

In [41]:
df_ami_out = df_ami.copy()
df_ami_out = df_ami_out.reset_index().rename(columns={"timestamp": "timestamp"})

df_ami_out.to_sql(
    name="ems_ess_core4",
    con=engine,
    if_exists="replace",
    index=False
)

0

In [43]:
# 다시 로드 (절대 다른 df와 intersection 하지 말 것)
PATH_AMI = "../data_csv/amiwea.csv"

df_ami = (
    pd.read_csv(PATH_AMI, parse_dates=["timestamp"])
    .sort_values("timestamp")
    .set_index("timestamp")
)

print(df_ami.shape)

(8760, 6)


In [44]:
df_ami_out = df_ami.reset_index()

df_ami_out.to_sql(
    name="ems_ess_core4",
    con=engine,
    if_exists="replace",
    index=False
)

8760

In [45]:
pd.read_sql(
    "SELECT COUNT(*) AS ems_rows FROM ems_ess_core4",
    engine
)

,ems_rows
0,8760


In [46]:
pd.read_sql("""
SELECT 
  (SELECT COUNT(*) FROM ems_ess_core4) AS ems_rows,
  (SELECT COUNT(*) FROM pacemaker_iot) AS pacemaker_rows;
""", engine)

,ems_rows,pacemaker_rows
0,8760,15000
